<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/metadata_filtering_company_name_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.74-cp310-cp310-linux_x86_64.whl size=61875621 sha256=1c5a798007f77fa71fa88d647372271e6dbdc2568123305fa7117a5c06e58f48
  Stored in directory: /root/.cache/pip/wheels/21/1f/6d/ee16805ea65ed00d89776a8ce82f3631340a43fc41edabed97
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf to /root/.cache/huggingface/hub/tmpgj1buli5
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [01:29<00:00, 85.5MB/s]
./models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.2 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [54]:
### Retrieval Grader

from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
template="""
  <|assistant|> You are tasked with identifying the correct spelling of the company name mentioned in the user's question by searching through the list of fixed company names in the database metadata.
  This precise spelling will be crucial for SQL filtering purposes. \n
  Provide a concise response containing only the correct company name. \n
  Please format your response as a JSON object with only a single key 'company', WITHOUT any additional commentary. <|end|>
  <|user|> Database metadata with company names: \n\n {metadata_list} \n\n User question: {question} <|end|>
  <|assistant|>
""",
input_variables=["question", "metadata_list"])

retrieval_grader = prompt | llm | JsonOutputParser()

In [35]:
!pip install sentence_transformers FlagEmbedding chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.6 MB

In [36]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'}, #gpu
    encode_kwargs=encode_kwargs
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
import chromadb
from langchain.storage.file_system import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.vectorstores import Chroma


persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports")
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [39]:
metadata = vectorstore.get()['metadatas']

In [47]:
metadata_list = []
for i in range(len(metadata)):
  metadata_list.append(metadata[i]['company'])
metadata_list = list(set(metadata_list))

In [48]:
metadata_list

['American Water Works Company, Inc.',
 'JOHNSON & JOHNSON',
 'LOCKHEED MARTIN CORP',
 'JPMORGAN CHASE & CO',
 'MGM Resorts International',
 'CORNING INC /NY',
 'COCA COLA CO',
 'NETFLIX INC',
 'MICROSOFT CORP',
 'PEPSICO INC',
 'Kraft Heinz Co',
 'ADVANCED MICRO DEVICES INC',
 'PayPal Holdings, Inc.',
 'ADOBE SYSTEMS INC',
 'Ulta Beauty, Inc.',
 'Activision Blizzard, Inc.',
 'VERIZON COMMUNICATIONS INC',
 'CVS HEALTH Corp',
 '3M CO',
 'AES CORP',
 'AMAZON COM INC',
 'FOOT LOCKER, INC.',
 'BEST BUY CO INC',
 'Amcor plc',
 'BOEING CO',
 'ADOBE INC.',
 'Square, Inc.',
 'GENERAL MILLS INC',
 'NIKE, Inc.',
 'NIKE INC',
 'PFIZER INC',
 'COSTCO WHOLESALE CORP /NEW',
 'AMERICAN EXPRESS CO',
 'Walmart Inc.']

In [55]:
retrieval_grader.invoke({"question": 'What was the Coca-Cola revenue in 2020?', "metadata_list": metadata_list})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =      12.85 ms /    18 runs   (    0.71 ms per token,  1401.21 tokens per second)
llama_print_timings: prompt eval time =    2435.59 ms /   453 tokens (    5.38 ms per token,   185.99 tokens per second)
llama_print_timings:        eval time =     656.19 ms /    17 runs   (   38.60 ms per token,    25.91 tokens per second)
llama_print_timings:       total time =    3399.57 ms /   470 tokens


{'company': 'COCA COLA CO'}

In [56]:
retrieval_grader.invoke({"question": "What is the paypal cost of goods sold (COGS) and how does the COGS compare to the total revenue?", "metadata_list": metadata_list})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =      12.43 ms /    22 runs   (    0.57 ms per token,  1769.77 tokens per second)
llama_print_timings: prompt eval time =     203.98 ms /    31 tokens (    6.58 ms per token,   151.98 tokens per second)
llama_print_timings:        eval time =     798.55 ms /    21 runs   (   38.03 ms per token,    26.30 tokens per second)
llama_print_timings:       total time =    1081.84 ms /    52 tokens


{'company': 'PayPal Holdings, Inc.'}

In [57]:
retrieval_grader.invoke({"question": "How does microsoft's management view the company performance?", "metadata_list": metadata_list})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =      12.12 ms /    21 runs   (    0.58 ms per token,  1732.24 tokens per second)
llama_print_timings: prompt eval time =     146.74 ms /    18 tokens (    8.15 ms per token,   122.67 tokens per second)
llama_print_timings:        eval time =     775.08 ms /    20 runs   (   38.75 ms per token,    25.80 tokens per second)
llama_print_timings:       total time =     996.58 ms /    38 tokens


{'company': 'MICROSOFT CORP'}

In [58]:
retrieval_grader.invoke({"question": "What are key risks mentioned in the 10-K and how does netflix plan to mitigate these risks?", "metadata_list": metadata_list})

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =      19.23 ms /    26 runs   (    0.74 ms per token,  1351.98 tokens per second)
llama_print_timings: prompt eval time =     256.27 ms /    34 tokens (    7.54 ms per token,   132.67 tokens per second)
llama_print_timings:        eval time =     963.60 ms /    25 runs   (   38.54 ms per token,    25.94 tokens per second)
llama_print_timings:       total time =    1380.44 ms /    59 tokens


{'company': 'NETFLIX INC'}